In [1]:
import os, json
import jsonlines

DATA_DIR = "../data"
data_list = [
    "KoAlpaca/KoAlpaca_v1.1.jsonl",
    "KoChatGPT/kochatgpt_1_SFT.jsonl",
    "korquad-chat-v1/korquad-chat.json",
    "OIG-small-chip2-ko/oig-smallchip2-dedu.jsonl",
    "ShareGPT_DeepL/ko_alpaca_style_dataset.json",
    "kullm_v2/kullm-v2.jsonl",
    "KoAlpaca/ko_alpaca_data.json"]

### KoAlpaca v1.0 dataset

In [44]:
data_path = data_list[-1]

with open(os.path.join(DATA_DIR, data_path), "r") as f:
    koalpaca_v1_0 = json.load(f)

len(koalpaca_v1_0)

49620

### KoAlpaca v1.1 dataset

In [2]:
data_path = data_list[0]

koalpaca_v1_1 = []
with jsonlines.open(os.path.join(DATA_DIR, data_path), "r") as f:
    for line in f.iter():
        koalpaca_v1_1.append(line)

len(koalpaca_v1_1)

21155

### KochatGPT SFT dataset

In [15]:
data_path = data_list[1]
with open(os.path.join(DATA_DIR, data_path), "r") as f:
    kochatgpt = json.load(f)

len(kochatgpt)

12000

### Korquad dataset

In [3]:
data_path = data_list[2]

korquad = []
with jsonlines.open(os.path.join(DATA_DIR, data_path), "r") as f:
    for line in f.iter():
        korquad.append(line)

len(korquad)

9619

### OIG dataset

In [17]:
data_path = data_list[3]

oig = []
with jsonlines.open(os.path.join(DATA_DIR, data_path), "r") as f:
    for line in f.iter():
        oig.append(line)

len(oig)

210282

### KULLM-v2 dataset

In [18]:
data_path = data_list[5]

kullm = []
with jsonlines.open(os.path.join(DATA_DIR, data_path), "r") as f:
    for line in f.iter():
        kullm.append(line)

len(kullm)

152630

### 6-th Science

In [4]:
import random
from collections import Counter

with open("../data/6th_grade_QA_과학_filtered.json", "r") as fin:
    six_sci_qa = json.load(fin)

keywords = []
for sample in six_sci_qa:
    keywords.append(sample["keywords"].split(",")[0])

most_keywords = Counter(keywords).most_common(32)

sci_six_res = []
key_qa_map = {}
for sample in six_sci_qa:
    for keyword, _ in most_keywords:
        if keyword in sample["keywords"]:
            if keyword not in key_qa_map:
                key_qa_map[keyword] = [sample]
            else:
                key_qa_map[keyword].append(sample)
            break
        else:
            inst = f"6학년 과학 문제:\n\n{sample['question']}"
            res = sample['answer']
            sci_six_res.append({"instruction": inst, "output": res})

inst_form = "{}에 대한 6학년 과학 주관식 문제 {}개 만들어줘"

sci_six_gen = []
for key, qas in key_qa_map.items():
    index = 0
    while len(qas) > index:      
        problems = []
        cur_len = 0
        choice = random.randint(1,3)
        for k, s in enumerate(qas[index:index+choice]):
            Q, A = s["question"], s["answer"]
            problem = f"문제 {k+1}: {Q}\n답변: {A}"
            cur_len += len(problem)
            problems.append(problem)
            if cur_len > 500:
                choice = k+1
                break
        
        inst = inst_form.format(key, choice)
        index += choice
        sci_six_gen.append({"instruction": inst, "output": "\n\n".join(problems)})

print(sci_six_res[0])
print(sci_six_gen[0])

{'instruction': '6학년 과학 문제:\n\n꽃의 구조와 기능은 무엇일까요?', 'output': '꽃은 꽃받침, 꽃잎, 수술, 암술로 이루어져 있습니다. 꽃의 구조는 이 네 부분으로 구성되어 있습니다.'}
{'instruction': '달 퍼즐에 대한 6학년 과학 문제 2개 만들어줘', 'output': '문제 1: 왜 달 퍼즐을 맞추는 활동을 하는 건가요?\n답변: 달 퍼즐 맞추기 활동을 통해 여러 가지 달의 모양에 대한 흥미와 호기심을 가질 수 있습니다.\n\n문제 2: 둥근 보름달 이외에 다른 모양의 달을 본 적이 있나요?\n답변: 여러 가지 모양의 달 퍼즐을 맞추어 보는 놀이를 통해 둥근 보름달 이외에도 다양한 모양의 달을 볼 수 있습니다.'}


### GSM8K_ko

In [5]:
import random

with open("../data/gsm8k_ko_main.json", "r") as fin:
    gsm8k = json.load(fin)

num_problem_gen = int(len(gsm8k) * 0.2)
random.shuffle(gsm8k)
gen_samples = gsm8k[:num_problem_gen]
gsm8k = gsm8k[num_problem_gen:]
inst = "사칙연산을 연습할 수 있도록 다단계 추론이 필요한 수학 주관식 문제 {}개 만들어줘. 그에 대한 해설과 정답도 같이 작성해줘."
gsm8k_gen = []
index = 0
while num_problem_gen > index:
    choice = random.randint(1,3)
    cur_len = 0
    problems = []
    for k, s in enumerate(gen_samples[index:index+choice]):
        Q, A = s["question"], s["answer"]
        problem = f"문제 {k+1}: {Q}\n해설: {A}"
        cur_len += len(problem)
        problems.append(problem)
        if cur_len > 500:
            choice = k+1
            break
        
    inst = inst.format(choice)
    index += choice
    gsm8k_gen.append({"instruction": inst, "output": "\n\n".join(problems)})

gsm8k_solve = []
for sample in gsm8k:
    gsm8k_solve.append({
        "instruction": sample["question"],
        "output": sample["answer"]
    })
        
print(len(gsm8k_gen), len(gsm8k_solve))

878 5913


### NAVERPEDIA_수업자료 생성 scenario

우리아이 성장백과 -> 영양 과목 \
어린이백과_사회, 어린이백과_한국사, 어린이백과_세계탐구, 어린이백과_유적/유물 탐구 -> 사회 과목 \
어린이백과_과학, 어린이백과_과학탐구 -> 과학 과목 \
어린이백과_세계사 -> 세계사 과목 \
어린이백과_수학 -> 수학 과목 \
어린이백과_소프트웨어 -> 컴퓨터 과목



In [6]:
#영양, 사회, 세계사, 수학, 과학, 컴퓨터
import os
from glob import glob
from tqdm import tqdm
import json

naverpedia_class_path = "../data/web_crawled/naverpedia_class.json"
#format for 성장백과, 사회, 한국사, 세계사, 수학, 과학 
ins_format = "{category} 내 {title}을 주제로 수업 자료를 작성해줘."
res_format = "{contents}"

with open(naverpedia_class_path, "r") as f:
    json_data = json.load(f)

naverpedia= []
for i in range(len(json_data)):
    ins = ins_format.format_map(json_data[i])
    res = json_data[i]["contents"].strip()
    naverpedia.append({"instruction":ins, "output":res})

In [7]:
print(naverpedia[-1])

{'instruction': '컴퓨터 과목 내 인터넷 중독 [Internet Addiction]을 주제로 수업 자료를 작성해줘.', 'output': '## 예시\n인터넷 중독 증상에 대하여 다음과 같은 인터넷 사이트를 통하여 진단할 수 있습니다.https://www.iapc.or.kr/dia/survey/addDiaSurveyNew.do?dia_type_cd=IABO만약 중독 증상을 보인다면 전문가와 상담하여 치료를 받는 것이 좋습니다. 중독 증상에 대하여 교육이나 상담을 할 수 있는 곳은 다음과 같습니다.스마트 쉼 센터 1599-0075## 설명\n인터넷에 중독되면 인터넷을 사용하지 않을 때 불안해지며 습관적으로 인터넷을 사용하게 됩니다. 또한 건강을 해치고 공부에 신경을 덜 쓰게 되며 친구 및 가족 관계에 소홀해지게 됩니다.'}


### NAVERPEDIA_manual하게 선별 필요 부분

+ 어린이백과_국어, 영어
+ 어린이백과_예체능 
+ 어린이백과_시사/논술
+ 어린이백과_문학 탐구
+ 어린이백과_인물 탐구
+ 어린이백과_지식e
+ 어린이백과_천재학습백과

In [8]:
naverpedia_path = "../data/web_crawled/naverpedia.json"
with open(naverpedia_path, "r") as f:
    json_data = json.load(f)

def get_dictionary_by_category(json, category):
    list = []
    for item in json:
        if "category" in item and item["category"] ==category:
            list.append(item)
    return list

#format for 어린이백과 시사/논술 
ins_format2 = "{title} 주제에 관련해서 실생활에서 볼 수 있는 사례를 들면서 설명하려고 해. {title}에 관한 사례를 들고 구체적으로 설명해줘."
res_format2 = "{contents}"
kidspedia_essay = get_dictionary_by_category(json_data, "어린이백과_시사/논술")

for i in range(len(kidspedia_essay)):
    ins = ins_format2.format_map(kidspedia_essay[i])
    res = res_format2.format_map(kidspedia_essay[i])
    naverpedia.append({"instruction":ins, "output":res})

#format for 어린이백과 국어
ins_format3 = "국어사전을 작성하려고 하는데 {title} 이라는 말의 뜻이나 실생활에서 사용하는 사례를 설명해줘."
res_format3 = "{contents}"
kidspedia_korean = get_dictionary_by_category(json_data, "어린이백과_국어")

for i in range(len(kidspedia_korean)):
    ins = ins_format3.format_map(kidspedia_korean[i])
    res = res_format3.format_map(kidspedia_korean[i])
    naverpedia.append({"instruction":ins, "output":res})

#format for 어린이백과 영어
ins_format4 = "{title} 주제로 영어 수업자료를 만들고 있어. 실 사례를 포함해줘."
res_format4 = "{contents}"
kidspedia_english = get_dictionary_by_category(json_data, "어린이백과_영어")

for i in range(len(kidspedia_english)):
    ins = ins_format4.format_map(kidspedia_english[i])
    res = res_format4.format_map(kidspedia_english[i])
    naverpedia.append({"instruction":ins, "output":res})

#format for 어린이백과 문학탐구
ins_format5 = "{title}를 주제로 어린이한테 교훈을 줄 수 있는 우화를 만들어줘."
res_format5 = "{contents}"
kidspedia_story = get_dictionary_by_category(json_data, "어린이백과_문학탐구")

for i in range(len(kidspedia_story)):
    ins = ins_format5.format_map(kidspedia_story[i])
    res = res_format5.format_map(kidspedia_story[i])
    naverpedia.append({"instruction":ins, "output":res})

#format for 어린이백과 인물탐구
ins_format6 = "{title} 인물의 생애와 업적을 소개하는 글을 작성해줘."
res_format6 = "{contents}"
kidspedia_celebrity = get_dictionary_by_category(json_data, "어린이백과_인물탐구")

for i in range(len(kidspedia_celebrity)):
    ins = ins_format6.format_map(kidspedia_celebrity[i])
    res = res_format6.format_map(kidspedia_celebrity[i])
    naverpedia.append({"instruction":ins, "output":res})

#format for 어린이백과 지식e
ins_format7 = "{title}을 주제로 수업을 진행하려는데 {title}에 대해 학생들이 흥미를 느끼도록 수업 자료를 작성해줘."
res_format7 = "{contents}"
kidspedia_knowledge = get_dictionary_by_category(json_data, "어린이백과_지식e")

for i in range(len(kidspedia_knowledge)):
    ins = ins_format7.format_map(kidspedia_knowledge[i])
    res = res_format7.format_map(kidspedia_knowledge[i])
    naverpedia.append({"instruction":ins, "output":res})

#format for 어린이백과 천재학습백과
ins_format8 = "{title}을 주제로 수업을 진행하는데 {title}에 관한 자료를 작성해줘."
res_format8 = "{contents}"
kidspedia_genius = get_dictionary_by_category(json_data, "어린이백과_천재학습백과")

for i in range(len(kidspedia_genius)):
    ins = ins_format8.format_map(kidspedia_genius[i])
    res = res_format8.format_map(kidspedia_genius[i])
    naverpedia.append({"instruction":ins, "output":res})

In [15]:
# print(koalpaca_v1_0[0])
print(koalpaca_v1_1[0])
# print(kochatgpt[0])
print(korquad[0])
# print(oig[0])
# print(sharegpt[0])
# print(kullm[-11])

{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?', 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.', 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268'}
{'source': 'korquad-chat', 'text': '<sys>1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악

In [11]:
import re
from tqdm import tqdm

def to_dialog(data, inst_key, out_key, domain, preprossing=True):
    dialog = []
    for sample in tqdm(data, total=len(data)):
        if preprossing:
            inst = re.sub(r"\?(\w)", r"?\n\1", sample[inst_key])
            output = re.sub(r"[.](\d+)[.]", r".\n\1.", sample[out_key])
        
        dialog.append(
            [{"from": "human", "value": inst},
             {"from": "bot", "value": output, "domain":domain}]
        )
    return dialog

print("Unifying except for Korquad dataset")
unified_data = []
# koalpacav1.1
unified_data.extend(to_dialog(koalpaca_v1_1, "instruction", "output", "general"))
# Naverpedia
unified_data.extend(to_dialog(naverpedia, "instruction", "output", "education"))
# GSM8K
unified_data.extend(to_dialog(gsm8k_solve, "instruction", "output", "education"))
unified_data.extend(to_dialog(gsm8k_gen, "instruction", "output", "education"))
# Sci 6 grade
unified_data.extend(to_dialog(sci_six_gen, "instruction", "output", "education"))
unified_data.extend(to_dialog(sci_six_res, "instruction", "output", "education"))

# kochatgpt
# unified_data.extend(to_dialog(kochatgpt, "prompt", "completion", "general"))
# OIG
# unified_data.extend(to_dialog(oig, "user_translated", "chip2_translated", "general"))
    
# KULLM-v2
"""
processed_kullm = []
for sample in tqdm(kullm, total=len(kullm)):
    if sample["input"]:
        inst = sample["instruction"].strip()+"\n\n"+sample["input"].strip()
    else:
        inst = sample["instruction"].strip()
        
    if len(inst) <= 15:
        continue
    
    output = re.sub(r"[.](\d+)[.]", r".\r\n\1.", sample["output"])
        
    processed_kullm.append(
        [{"from": "human", "value": inst},
         {"from": "bot", "value": output}]
    )
print(len(processed_kullm))
unified_data.extend(processed_kullm)
"""

# unified_data.extend(koalpaca_v1_0)
len(unified_data)

Unifying except for Korquad dataset


  0%|          | 0/21155 [00:00<?, ?it/s]

100%|██████████| 26782/26782 [00:00<00:00, 136704.15it/s]


57749

### Processing Korquard dataset

In [12]:
for sample in korquad:#, total=len(korquad)):
    try:
        turns = sample["text"].split("\n")
        context = turns[0].split(">")[1].strip()
        dialog = []
        for turn in turns[1:]:
            splited = turn.split(">")
            role = splited[0][1:]
            text = splited[1].strip()
            dialog.append({"role": role, "text": text})
            
        assert dialog[0]["role"]=="usr" and len(dialog)%2==0
        
        inst_res = []
        for i in range(0, len(dialog), 2):
            user = dialog[i]["text"]
            if i==0:
                user = context.strip()+"\n\n"+user.strip()
            bot = dialog[i+1]["text"]
        
            inst_res.extend(
                [{"from": "human", "value": user},
                 {"from": "bot", "value": bot,}]
            )
        inst_res[1]["domain"] = "general"
                
        unified_data.append(inst_res)
    except:
        pass
    
len(unified_data)

66977

### AI Hub Book summarization

In [13]:
import os
from glob import glob
from tqdm import tqdm

json_files = glob("../data/AIhub_book_summary/*.json")

ins1_format = "다음 문단은 \'{kdc_label}\'로 분류된 문서의 일부분이다.\n\n{passage1}\n\n이 문단에 이어질 한 문장을 작성해줘"
res1_format = "\"{passage2}\"가 될 수 있겠습니다."
ins2_format = "완성된 문단을 간단히 요약하고 해당 문서의 제목을 지어줘"
res2_format = "이 문단은 다음과 같이 요약될 수 있습니다.\r\n- 요약: {summary}\r\n\r\n- {kdc_label} {doc_type} 제목 제안: \"{doc_name}\""

target_kdc = ["보건의료", "체육", "보육·가족및여성", "한국문학", "한국어", "문학", "문화재", "역사", "음악", "심리학"]
book_summary = []
kdcs = []
for path in tqdm(json_files):
    with open(path, "r") as f:
        json_data = json.load(f)
    data = json_data["metadata"]
    kdcs.append(data["kdc_label"])
        
    if "교육" in data["kdc_label"] or data["kdc_label"] in target_kdc:
        splited_passage = json_data["passage"].split(". ")
        data["passage1"] = ". ".join(splited_passage[:-1])+"."
        data["passage2"] = splited_passage[-1]
        data["summary"] = json_data["summary"]
        ins1 = ins1_format.format_map(data)
        res1 = res1_format.format_map(data)
        ins2 = ins2_format.format_map(data)
        res2 = res2_format.format_map(data)
        book_summary.append([
            {"from":"human", "value":ins1},
            {"from":"bot", "value":res1, "domain":"general"},
            {"from":"human", "value":ins2},
            {"from":"bot", "value":res2},
        ])
    #     data["passage"] = json_data["passage"]
    #     data["summary"] = json_data["summary"]
    #     ins = ins_format.format_map(data)
    #     res = res_format.format_map(data)
    #     kdcs.append(data["kdc_label"])
    #     book_summary.append({"instruction":ins, "output":res})

100%|██████████| 180001/180001 [00:06<00:00, 27881.20it/s]


In [14]:
unified_data.extend(book_summary)
len(unified_data)

99500

### AI hub 주제별 일상 대화 데이터

In [24]:
from glob import glob
import json

chitchat_list = glob("../data/AIhub_chitchat/*.json")
chitchat_data = []

for path in chitchat_list:
    try:
        with open(path, "r") as fin:
            chitchat_data.append(json.load(fin))
    except Exception:
        print(path)
        continue

len(chitchat_data)

../data/AIhub_chitchat/KAKAO_1648_13.json


98651

In [25]:
from tqdm import tqdm
import re

dialogs = []
for data in chitchat_data:
    data = data["info"][0]
    domain = data["annotations"]["speaker_type"]
    if data["annotations"]["speaker_type"]!="1:1":
        continue
    
    data = data["annotations"]
    lines = data["lines"]
    
    if len(lines) < 2 or not data["lines"][0]["norm_text"]:
        continue
    
    if len(set([line["speaker"]["id"] for line in lines]))!=2:
        continue
       
    user_info = lines[0]["speaker"]
    u_id = user_info["id"]
    u_gender = user_info["sex"]
    u_age = user_info["age"]
    
    prev_id = u_id
    dialog = []
    user = []
    bot = []
    for i in range(len(lines)):
        cur_id = lines[i]["speaker"]["id"]
        if prev_id!=cur_id and prev_id!=u_id:
            response = " ".join(bot).strip()
            response = re.sub("키키", "ㅋㅋ", response)
            dialog.extend(
                [{"from": "human", "value": " ".join(user).strip()},
                 {"persona": {"age": a_age, "gender": a_gender, "domain": "일상 대화"}, "from": "bot", "value": response}]
            )
            user = []
            bot = []

        if cur_id!=u_id:
            a_gender = lines[i]["speaker"]["sex"]
            a_age = lines[i]["speaker"]["age"]
            bot.append(lines[i]["norm_text"])
        else:
            user.append(lines[i]["norm_text"])
            
        prev_id = cur_id
    
    if dialog:
        dialogs.append(dialog.copy())
    
print(len(dialogs))

unified_data.extend(dialogs)

70613


In [15]:
len(unified_data)

99500

In [16]:
import json

with open("../data/unified_instruction_clean.json", "w", encoding="utf-8") as f:
    json.dump(unified_data, f, ensure_ascii=False) # ensure_ascii로 한글이 깨지지 않게 저장